#importing basic packages

In [ ]:
import re
from io import StringIO
from datetime import datetime, timedelta
import requests
import pandas as pd

In [ ]:
# class for scraping yahoo data
class YahooFinanceHistory:
    timeout = 2
    crumb_link = 'https://finance.yahoo.com/quote/{0}/history?p={0}'
    crumble_regex = r'CrumbStore":{"crumb":"(.*?)"}'
    quote_link = 'https://query1.finance.yahoo.com/v7/finance/download/{quote}?period1={dfrom}&period2={dto}&interval=1d&events=history&crumb={crumb}'



In [ ]:
     # class variables for the api url request
    def __init__(self, symbol, days_back=7):
        self.symbol = symbol
        self.session = requests.Session()
        self.dt = timedelta(days=days_back)


In [ ]:
    def get_crumb(self):
    response = self.session.get(self.crumb_link.format(self.symbol), timeout=self.timeout)
    response.raise_for_status()
    match = re.search(self.crumble_regex, response.text)
    if not match:
        raise ValueError('Could not get crumb from Yahoo Finance')
    else:
        self.crumb = match.group(1)

In [ ]:
    def get_quote(self):
        if not hasattr(self, 'crumb') or len(self.session.cookies) == 0:
            self.get_crumb()
        now = datetime.utcnow()
        dateto = int(now.timestamp())
        datefrom = int((now - self.dt).timestamp())
        url = self.quote_link.format(quote=self.symbol, dfrom=datefrom, dto=dateto, crumb=self.crumb)
        response = self.session.get(url)
        response.raise_for_status()
        read_csv = pd.read_csv(StringIO(response.text), parse_dates=['Date'])
        read_csv.to_csv('stocks_data_30_8.csv', index=False)